## Pandas实现数据合并concat

### 使用场景：

批量合并形同格式的Excel、给DataFrame添加行、给DataFrame添加列

### concat语法：

使用某种合并的方式（inner/outer），沿着某个轴向（axis=1/0）,把多个Pandas对象（DataFrame/Series）合并成一个  
#### pandas.concat(objs,axis=0,join='outer',ignore_index=False)  
1. objs:一个列表，元素可以是DataFrame或者Series，可以混合  
2. axis:默认是0代表行合并，如果等于1代表按列合并  
3. join:合并的时候索引的对齐方式，默认是outer join，也可以是innner join  
4. ignore_index:是否忽略掉原来的数据索引

### append语法：

append只有按行合并，没有按列合并，相当于concat按行的简写形式

#### DataFrame.append(other,ignore_index=False)

1. other:单个dataframe、series、dict或者列表
2. ignore_index:是否忽略原来的数据索引

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
df1 = pd.DataFrame({
    'A':['A0','A1','A2','A3'],
    'B':['B0','B1','B2','B3'],
    'C':['C0','C1','C2','C3'],
    'D':['D0','D1','D2','D3']
})
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [3]:
df2 = pd.DataFrame({
    'A':['A4','A5','A6','A7'],
    'B':['B4','B5','B6','B7'],
    'C':['C4','C5','C6','C7'],
    'E':['E4','E5','E6','E7']
})
df2

,A,B,C,E
0,A4,B4,C4,E4
1,A5,B5,C5,E5
2,A6,B6,C6,E6
3,A7,B7,C7,E7


In [4]:
# 默认参数：axis=0，join=outer,ignore_index=False
pd.concat([df1,df2])   # 每一行进行链接，outer模式补齐列索引

,A,B,C,D,E
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
0,A4,B4,C4,NaN,E4
1,A5,B5,C5,NaN,E5
2,A6,B6,C6,NaN,E6
3,A7,B7,C7,NaN,E7


In [5]:
pd.concat([df1,df2],join='inner',ignore_index=True)  # 每一行连接，inner模式保留每行数据都有的列索引,过滤不匹配的列

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3
4,A4,B4,C4
5,A5,B5,C5
6,A6,B6,C6
7,A7,B7,C7


In [6]:
pd.concat([df1,df2],axis=1)  # 每一列连接，使用axis=1相当于添加新列

,A,B,C,D,A,B,C,E
0,A0,B0,C0,D0,A4,B4,C4,E4
1,A1,B1,C1,D1,A5,B5,C5,E5
2,A2,B2,C2,D2,A6,B6,C6,E6
3,A3,B3,C3,D3,A7,B7,C7,E7


In [8]:
# 添加一列Series
s1 = pd.Series(list(range(4)),name="E")
pd.concat([df2,s1],axis=1)   # 列表是DataFrame和Series的混合

,A,B,C,E,E
0,A4,B4,C4,E4,0
1,A5,B5,C5,E5,1
2,A6,B6,C6,E6,2
3,A7,B7,C7,E7,3


In [13]:
# 添加多列
s2 = df1.apply(lambda x:x['A']+'_GG',axis=1)
s2.name="G"
pd.concat([df1,s1,s2],axis=1)

,A,B,C,D,E,G
0,A0,B0,C0,D0,0,A0_GG
1,A1,B1,C1,D1,1,A1_GG
2,A2,B2,C2,D2,2,A2_GG
3,A3,B3,C3,D3,3,A3_GG


In [14]:
# 使用DataFrame.append:给一个dataframe添加另外一个dataframe
df1 = pd.DataFrame([[1,2],[3,4]],columns=list("AB"))
df1

,A,B
0,1,2
1,3,4


In [19]:
df2 = pd.DataFrame([[5,6],[7,8]],columns=list("AB"))
df2

,A,B
0,5,6
1,7,8


In [20]:
df1.append(df2,ignore_index=True)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8


In [22]:
# 如何一行一行添加数据
# 低性能版本
df = pd.DataFrame(columns=['A'])
for i in range(5):
    # 这里每一次执行都会复制df
    df = df.append({'A':i},ignore_index=True)
df

,A
0,0
1,1
2,2
3,3
4,4


In [24]:
# 性能较好版本
pd.concat(
    [pd.DataFrame([i],columns=["A"]) for i in range(5)],
    ignore_index = True
)

,A
0,0
1,1
2,2
3,3
4,4
